In [180]:
import pandas as pd
import numpy as np
import time
from datetime import datetime as dt, date, timedelta
from polygon import RESTClient
from secret import API_KEY

In [181]:
# Getting API key and client
api_key = API_KEY
client = RESTClient(api_key)

In [182]:
# Importing buy/sell signals
data = pd.read_csv('signals.csv')

In [183]:
# Creating columns for prices purchased and sold at 
data['call_prices'] = np.zeros(len(data))
data['put_prices'] = np.zeros(len(data))

In [ ]:
# Setting ticker
underlying_ticker = 'AAPL'

In [333]:
def get_option_ticker(timestamp, price, type):
    # Fixing timestamps
    date = pd.to_datetime(timestamp).date()
    date_more = date + timedelta(weeks=1)

    try:
        # Initialize a list to collect all contract data
        options_list = []

        # Fetch options data
        options_data = client.list_options_contracts(
            underlying_ticker=underlying_ticker,
            expiration_date_lte=date_more,
            expiration_date_gte=date,
            strike_price_gte=(price),
            strike_price_lte=(price+1),
            contract_type=type,
            limit=1000,
            expired=True
        )

        # Loop through the options data and append each contract's data to options_list
        for option in options_data:
            options_list.append(option.__dict__)

        # Convert the list of options data to a DataFrame
        df = pd.DataFrame(options_list)

        # Finding closest ticker
        option_ticker = df['ticker'].iloc[0]

        return option_ticker
    except Exception as e:
        print(f'Error: {e}')

    return None

def get_option_data(timestamp, ticker):
    date = pd.to_datetime(timestamp).date()

    try:
        aggs = client.get_aggs(
                ticker=ticker,
                multiplier=1,
                timespan="minute",
                from_=date,
                to=date,
                limit=5000
            )
        
        df = pd.DataFrame([agg.__dict__ for agg in aggs])

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')

        df['time'] = df['timestamp'].dt.time
        df['time'] = df['time'].apply(lambda t: t.strftime("%H:%M:%S"))
        
        return df
    except Exception as e:
        print(f"Error: {e}")

    return None


In [331]:
ticker = get_option_ticker(data.at[0, 'timestamp'], data.at[0, 'close'], 'call')

df = get_option_data(data.at[0, 'timestamp'], ticker)

In [ ]:
for index, row in data.head(4).iterrows():
    if row['call_buy']:
        timestamp = row['timestamp']
        ticker = get_option_ticker(timestamp, row['close'], 'call')
        option_price_data = get_option_data(timestamp, ticker)

        try:
            buy_call_price = (option_price_data[option_price_data['time'] == row['time']]).at[0, 'close']
        except:
            print('NO DATA')

    elif row['call_sell']:
        try:
            sell_call_price = (option_price_data[option_price_data['time'] == row['time']]).at[0, 'close']
        except:
            print('NO DATA')

    elif row['put_buy']:
        timestamp = row['timestamp']
        ticker = get_option_ticker(timestamp, row['close'], 'put')
        option_price_data = get_option_data(timestamp, ticker)

        try:
            buy_put_price = (option_price_data[option_price_data['time'] == row['time']]).at[0, 'close']
        except:
            print('NO DATA')

    elif row['put_sell']:
        try:
            sell_put_price = (option_price_data[option_price_data['time'] == row['time']]).at[0, 'close']
        except:
            print('NO DATA')

KeyError: 0